In [1]:
from glotaran_tools.specification_parser import parse_yml
from glotaran_core.model import Dataset
from glotaran_models.kinetic import KineticSeperableModel
import numpy as np
from lmfit import Parameters
# Imports for plotting

import matplotlib.pyplot as plt
%matplotlib notebook

fitspec = '''
type: kinetic

parameter: {}

compartments: [s1, s2, s3]

initial_concentrations:
    - label: i1
      parameter: [4,5,6]

megacomplexes:
    - label: mc1
      k_matrices: [k1]

k_matrices:
  - label: "k1"
    matrix: {{
      '("s1","s1")': 1,
      '("s2","s2")': 2,
      '("s3","s3")': 3,
}}

irf: []

datasets: 
  - label: dataset1
    type: spectral
    initial_concentration: i1
    megacomplexes: [mc1]
    path: 'data/dataset1.ascii'

'''
   

In [10]:
test_parameter = [101e-4, 202e-5, 505e-6, 80, 200, 300]
test_amps = [1,2,3]
times = np.asarray(np.arange(0, 1500, 1.5))
simparams = Parameters()
simparams.add("p1", 101e-3)
simparams.add("p2",202e-4)
simparams.add("p3",505e-5)
simparams.add("p4",20)
simparams.add("p5",300)
simparams.add("p6",200)

In [11]:
        
model = parse_yml(fitspec.format(test_parameter))

In [12]:
fitmodel = KineticSeperableModel(model)

In [13]:
data = fitmodel.eval(simparams, *times, **{'dataset':'dataset1','amplitudes':[10, 200, 3000]})

In [14]:
c = fitmodel.c_matrix(fitmodel.get_initial_fitting_parameter().valuesdict(), *times)
print(c.shape)

(1000, 3)


In [15]:
result = fitmodel.fit(fitmodel.get_initial_fitting_parameter(), 
                      *times, 
                      **{"data": data}
                     )
print(result.params)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.4004e+04                                    9.70e+06    
       1             10         1.9183e+04      4.82e+03       5.64e-03       6.34e+06    
       2             11         4.1407e+03      1.50e+04       5.64e-03       7.47e+06    
       3             12         6.0917e+02      3.53e+03       1.13e-02       2.38e+06    
       4             13         1.5118e+02      4.58e+02       2.25e-02       1.46e+05    
       5             14         1.4474e+02      6.44e+00       4.51e-02       1.42e+05    
       6             18         1.4273e+02      2.01e+00       1.41e-03       7.74e+04    
       7             19         1.2676e+02      1.60e+01       3.52e-04       1.44e+04    
       8             20         1.2336e+02      3.40e+00       7.04e-04       3.29e+04    
       9             21         1.1616e+02      7.20e+00       7.04e-04       2.28e+04    